[![Jupyter Notebook](https://img.shields.io/badge/Source%20on%20GitHub-orange)](https://github.com/laminlabs/lamindb/blob/main/docs/arrays.ipynb)

# Query arrays

We saw how LaminDB allows to query & search across artifacts & collections using registries: {doc}`registries`.

Let us now look at the following case:

```
# get a lookup for labels
ulabels = ln.ULabel.lookup()
# query a parquet file matching an "setosa"
df = ln.Artifact.filter(ulabels=ulabels.setosa, suffix=".suffix").first().load()
# query all observations in the DataFrame matching "setosa"
df_setosa = df.loc[:, df.iris_organism_name == ulabels.setosa.name]  
```

Because the artifact was validated, querying the `DataFrame` is guaranteed to succeed!

Such within-collection queries are also possible for cloud-backed collections using DuckDB, TileDB, zarr, HDF5, parquet, and other storage backends.

- For a use case with TileDB, see: {doc}`docs:cellxgene`
- For a use case with DuckDB, see: {doc}`docs:rxrx`

In this notebook, we show how to subset an `AnnData` and generic `HDF5` and `zarr` collections accessed in the cloud.

In [ ]:
!lamin login testuser1
!lamin init --storage s3://lamindb-ci/test-array-notebook --name test-array-notebook

In [ ]:
import lamindb as ln

In [ ]:
ln.settings.verbosity = "info"

We'll need some test data:

In [ ]:
ln.Artifact("s3://lamindb-ci/lndb-storage/pbmc68k.h5ad").save()
ln.Artifact("s3://lamindb-ci/lndb-storage/testfile.hdf5").save()

## AnnData

An `h5ad` artifact stored on s3:

In [ ]:
artifact = ln.Artifact.get(key="lndb-storage/pbmc68k.h5ad")

In [ ]:
artifact.path

In [ ]:
adata = artifact.open()

This object is an `AnnDataAccessor` object, an `AnnData` object backed in the cloud:

In [ ]:
adata

Without subsetting, the `AnnDataAccessor` object references underlying lazy `h5` or `zarr` arrays:

In [ ]:
adata.X

You can subset it like a normal `AnnData` object:

In [ ]:
obs_idx = adata.obs.cell_type.isin(["Dendritic cells", "CD14+ Monocytes"]) & (
    adata.obs.percent_mito <= 0.05
)
adata_subset = adata[obs_idx]
adata_subset

Subsets load arrays into memory upon direct access:

In [ ]:
adata_subset.X

To load the entire subset into memory as an actual `AnnData` object, use `to_memory()`:

In [ ]:
adata_subset.to_memory()

## Generic HDF5

Let us query a generic HDF5 artifact:

In [ ]:
artifact = ln.Artifact.get(key="lndb-storage/testfile.hdf5")

And get a backed accessor:

In [ ]:
backed = artifact.open()

The returned object contains the `.connection` and `h5py.File` or `zarr.Group` in `.storage`

In [ ]:
backed

In [ ]:
backed.storage

In [ ]:
# clean up test instance
!lamin delete --force test-array-notebook